In [2]:
from collections import Counter
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import math
from scipy import optimize
#from ipywidgets import interact
#import ipywidgets as wg
import statistics as st

# Funções para carregar vetores com hits e PMTs

In [3]:
#Função para carregar um vetor de hits / PMT 
def fnt_array_hitspmt(list_sum):
    xf = []
    for filetr in list_sum:

         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            if (i_PMT != 0):
                xf.append(float(i_Hits / i_PMT))
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    if (i_PMT != 0):
                        xf.append(float(i_Hits / i_PMT))
    fo.close()
    return(xf)

#Função para carregar um vetor de dois vetores: 1) hits; 2) PMT; 3) Energia
#novo parametro trigger_pmt para somente considerar eventos com multiplicidade >=trigger_pmt
#a trigger no detector somente ativa as NDAQs se mais que 5 PMTs fizeram a detecção - 31/05/2020
#valor padrão trigger_pmt=0 para considerar tudo
def fnt_array_vet_hits_pmt(list_sum,trigger_pmt=0):
    xf = []
    yf = []
    zf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            f_Energy = ( i_Hits * 77.96 ) * 4.84E-4 + 0.620 #float(s_energy[43:63])
            #DUQ/hits = 77.96 e Energia(MeV) = Hits * (DUQ/hits) * 4.9E-4 + 0.542 
            #i_HitsPMT = float(i_Hits / i_HitsPMT
            if (i_Hits != 0)and(i_PMT>=trigger_pmt)and(i_Hits<=250):
                yf.append(i_PMT)
                xf.append(i_Hits)
                zf.append(f_Energy)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_PMT = int(s_energy[15:21])
                    #print(energy_histo)
                    i_Hits = int(s_energy[8:14])
                    f_Energy = ( i_Hits * 77.96 ) * 4.84E-4 + 0.620
                    #print(i_Hits,fEner)
                    #DUQ/hits = 77.96 e Energia(MeV) = Hits * (DUQ/hits) * 4.9E-4 + 0.542 
                    #i_HitsPMT = int(s_energy[8:14])
                    if (i_Hits != 0)and(i_PMT>=trigger_pmt)and(i_Hits<=250):
                        yf.append(i_PMT)
                        xf.append(i_Hits)
                        zf.append(f_Energy)
    fo.close()
    return( [xf, yf, zf] )

#Função para carregar um vetor de dois vetores: 1) hits; 2) PMT 
def fnt_array_vet_hits_pmt_regime(list_sum, regime1, regime2):
    xf = []
    yf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            #i_HitsPMT = float(i_Hits / i_HitsPMT
            if (i_Hits != 0)and(i_PMT>regime1)and(i_PMT<=regime2):
                yf.append(i_PMT)
                xf.append(i_Hits)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != "")and(i_PMT>regime1)and(i_PMT<=regime2):
                    i_PMT = int(s_energy[15:21])
                    #print(energy_histo)
                    i_Hits = int(s_energy[8:14])
                    #i_HitsPMT = int(s_energy[8:14])
                    if (i_Hits != 0):
                        yf.append(i_PMT)
                        xf.append(i_Hits)
    fo.close()
    return( [xf, yf] )

#Função para carregar um vetor de hits / PMT  com limite maior e menor de uma determinada multiplicidade de PMT
def fnt_array_hitspmt_limits(list_sum, check_limit_PMT):
    xf = [] #menor multiplicidade
    yf = [] #maior multiplicidade
    for filetr in list_sum:
         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
#            fg = open(list_gama[0],'r')
#            s_gama = fg.readline().replace(",",".")
            if (i_PMT != 0):
                if (i_PMT > check_limit_PMT):
                    yf.append(float(i_Hits / i_PMT))
#                    print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                else:
                    xf.append(float(i_Hits / i_PMT))
#                    print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
#                s_gama = fg.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    if (i_PMT != 0):
                        if (i_PMT > check_limit_PMT):
                            yf.append(float(i_Hits / i_PMT))
#                            print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                        else:
                            xf.append(float(i_Hits / i_PMT))
#                            print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
    fo.close()
    return([xf,yf])

#Função para carregar um vetor de hits / PMT  com limite maior e menor de uma determinada multiplicidade de PMT
#O diferencial desta função é que aqui estou comparando energia por multiplicidade
def fnt_array_hitspmt_limits_2(list_sum, check_limit_PMT, list_gama):
    xf = [] #menor multiplicidade
    yf = [] #maior multiplicidade
    xg = []
    yg = []
    af = []
    bf = []
    for filetr in list_sum:
         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            fg = open(list_gama[0],'r')
            s_gama = fg.readline().replace(",",".")
            f_gama = float(s_energy[22:36])  #para converter em MeV
            if (i_PMT != 0):
                if (i_PMT > check_limit_PMT):
                    yf.append(float(i_Hits / i_PMT))
                    yg.append(float(s_gama[37:]))
                    bf.append(f_gama)
                    #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                else:
                    xf.append(float(i_Hits / i_PMT))
                    xg.append(float(s_gama[37:]))
                    af.append(f_gama)
                    #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                s_gama = fg.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    f_gama = float(s_energy[22:36])  #para converter em MeV
                    if (i_PMT != 0):
                        if (i_PMT > check_limit_PMT):
                            yf.append(float(i_Hits / i_PMT))
                            yg.append(float(s_gama[37:]))
                            bf.append(f_gama)
                            #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                        else:
                            xf.append(float(i_Hits / i_PMT))
                            xg.append(float(s_gama[37:]))
                            af.append(f_gama)
                            #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
    fo.close()
    return([xf,yf,xg,yg,af,bf])

#Função para carregar um vetor de hits / PMT  com limite maior e menor de uma determinada multiplicidade de PMT
#O diferencial desta função é que aqui estou comparando energia por multiplicidade
def fnt_array_hitspmt_limits_3(list_sum, check_limit_PMT):
    xf = [] #menor multiplicidade
    yf = [] #maior multiplicidade
    xg = []
    yg = []
    for filetr in list_sum:
         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            #fg = open(list_gama[0],'r')
            #f_gama = float(s_energy[22:41])*1000  #para converter em MeV
            f_gama = float(s_energy[22:36])  #para converter em MeV
            if (i_PMT != 0):
                if (i_PMT > check_limit_PMT):
                    yf.append(float(i_Hits / i_PMT))
                    yg.append(f_gama)
                    #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                else:
                    xf.append(float(i_Hits / i_PMT))
                    xg.append(f_gama)
                    #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                #s_gama = fg.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    #f_gama = float(s_energy[22:41])*1000 #para converter em MeV
                    f_gama = float(s_energy[22:36]) #para converter em MeV
                    if (i_PMT != 0):
                        if (i_PMT > check_limit_PMT):
                            yf.append(float(i_Hits / i_PMT))
                            yg.append(f_gama)
                            #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                        else:
                            xf.append(float(i_Hits / i_PMT))
                            xg.append(f_gama)
                            #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
    fo.close()
    return([xf,yf,xg,yg])

#Função para carregar um vetor de dois vetores: 1) hits; 2) PMT; 3) Energia Depositada; 4) Energia coletada
def fnt_array_vet_hits_energy(list_sum):
    xf = []
    yf = []
    chf = []
    ehf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            f_Cherenkov = float(s_energy[22:42])*1000
            f_EnergyHit = float(s_energy[43:63])*1000
            if (i_Hits != 0):
                yf.append(i_PMT)
                xf.append(i_Hits)
                chf.append(f_Cherenkov)
                ehf.append(f_EnergyHit)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    f_Cherenkov = float(s_energy[22:42])*1000
                    f_EnergyHit = float(s_energy[43:63])*1000
                    if (i_Hits != 0):
                        yf.append(i_PMT)
                        xf.append(i_Hits)
                        chf.append(f_Cherenkov)
                        ehf.append(f_EnergyHit)
    fo.close()
    return( [xf, yf, chf, ehf] )


In [2]:
def fnt_array_scatter_hits(list_sum):
    xf = []
    yf = []
    for filetr in list_sum:
        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            if (i_Hits != 0):
                xf.append(i_PMT)
                yf.append(i_Hits)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_PMT = int(s_energy[15:21])
                    #print(energy_histo)
                    i_Hits = int(s_energy[8:14])
                    if (i_Hits != 0):
                        xf.append(i_PMT)
                        yf.append(i_Hits)
        fo.close()

    return([xf,yf])